# Basic OpenAI Agents SDK
- Basic SDK usage with evals
- Use langfuse for prompt repository, evaluation, observability, user feedback
- Run batches of prompt async against lists and dataframes


In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import langfuse
from langfuse import get_client
from langfuse import Langfuse
from langfuse.openai import openai
# from langfuse.openai import AsyncOpenAI
import logfire
from llm import LangfuseClient

from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent  # methods to apply prompts async to large batches
from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent



In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")
print(f"LangFuse           {langfuse.version.__version__}")
print(f"Logfire            {logfire.__version__}")


OpenAI:            1.107.0
OpenAI Agents SDK  0.2.11
Pydantic           2.11.7
LangFuse           3.3.4
Logfire            4.7.0


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(logging.INFO)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(logging.INFO)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

21:42:51 | NewsletterAgent.newsletter_agent | INFO | Test info message
21:42:51 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
21:42:51 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

In [5]:
# Configure logfire instrumentation.
# OpenAI used logfire from Pydantic AI and we need this for langfuse to handle some traces sent automatically by Agents SDK
logfire.configure(
    service_name="my_agent_service", 
    send_to_logfire=False,
    console=False,  # Disable console output
)
logging.getLogger("logfire").setLevel(logging.WARNING)
# Set logfire logger to WARNING level to reduce output
logfire.instrument_openai_agents()

# Langfuse Prompt Repo

In [6]:
# initialize langfuse for observability
# git clone https://github.com/langfuse/langfuse.git
# cd langfuse
# go to localhost:3000
# set up an org, project, get API keys and put in .env

lf_client = get_client()
 
# Verify connection
if lf_client.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")# Get production prompts
prompt = lf_client.get_prompt("newsagent/headline_classifier")

# Get prompt from repository by label
# You can use as many labels as you'd like to identify different deployment targets
prompt = lf_client.get_prompt("newsagent/headline_classifier", label="production")
print(prompt.prompt, "\n")
prompt = lf_client.get_prompt("newsagent/headline_classifier", label="latest")
print(prompt.prompt, "\n")

# Get by version number, usually not recommended as it requires code changes to deploy new prompt versions
prompt = lf_client.get_prompt("newsagent/headline_classifier", version=1)
print(prompt.prompt, "\n")


Langfuse client is authenticated and ready!
[{'type': 'message', 'role': 'system', 'content': 'You are a content-classification assistant that labels news headlines as AI-related or not.\nReturn JSON that matches the provided schema\n\nA headline is AI-related if it mentions (explicitly or implicitly):\n- Core AI models: machine learning, neural / deep / transformer networks\n- AI Applications: computer vision, NLP, robotics, autonomous driving, generative media\n- AI hardware, GPU chip supply, AI data centers and infrastructure\n- Companies or labs known for AI: OpenAI, DeepMind, Anthropic, xAI, NVIDIA, etc.\n- AI models & products: GPT-5, Gemini, Claude, Midjourney, DeepSeek, etc.\n- New AI products and AI integration into existing products/services\n- AI policy / ethics / safety / regulation / analysis\n- Research results related to AI\n- AI industry figures (Sam Altman, Demis Hassabis, Dario Amodei, etc.)\n- AI market and business developments, funding rounds, partnerships centered

In [7]:
system_prompt, user_prompt, model = LangfuseClient().get_prompt("newsagent/headline_classifier")
print("system prompt")
print(system_prompt)
print() 

print("user prompt")
print(user_prompt)
print() 

print("model")
print(model)

INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/headline_classifier' from Langfuse
INFO:llm:Parsed prompt 'newsagent/headline_classifier': model=gpt-5-mini, system_len=1093, user_len=48


system prompt
You are a content-classification assistant that labels news headlines as AI-related or not.
Return JSON that matches the provided schema

A headline is AI-related if it mentions (explicitly or implicitly):
- Core AI models: machine learning, neural / deep / transformer networks
- AI Applications: computer vision, NLP, robotics, autonomous driving, generative media
- AI hardware, GPU chip supply, AI data centers and infrastructure
- Companies or labs known for AI: OpenAI, DeepMind, Anthropic, xAI, NVIDIA, etc.
- AI models & products: GPT-5, Gemini, Claude, Midjourney, DeepSeek, etc.
- New AI products and AI integration into existing products/services
- AI policy / ethics / safety / regulation / analysis
- Research results related to AI
- AI industry figures (Sam Altman, Demis Hassabis, Dario Amodei, etc.)
- AI market and business developments, funding rounds, partnerships centered on AI
- Any other news with a significant AI component

Not AI-related: business software, cr

# Basic usage
- Run a prompt using agents
- Sessions
- Route through Langfuse for observability
- Save logs
- View traces and evals


In [8]:
# Get current `production` version of the prompt via raw langfuse client
system_prompt = lf_client.get_prompt("swallow/system")
 
# Insert variables into prompt template
# compiled_prompt = prompt.compile(criticlevel="expert", movie="Dune 2")

system_prompt.prompt


'You are an expert on airspeed velocities of swallows. Answer questions about swallow flight speeds with authority and humor when appropriate.'

In [9]:
# delete this local db to store agent sessions 
[os.remove(f) for f in glob.glob('swallow.db*') if os.path.exists(f)]


[None, None, None]

In [10]:
# run a chat using the OpenAI Agent class with a persistent session (and through langfuse for observability)

user_prompt = lf_client.get_prompt("swallow/user1").compile()
print(user_prompt)

openai_client = AsyncOpenAI()

# async def main():
agent = Agent(
    name="Assistant",
    instructions=system_prompt.prompt,
    model=OpenAIResponsesModel(model="gpt-4.1", openai_client=openai_client),
)

# 1) Create (or reuse) a session. Use a durable DB path if you want persistence.
session = SQLiteSession("test_swallow_chat", "swallow.db")

result = await Runner.run(agent, user_prompt, session=session)
display(Markdown(result.final_output))

# loop = asyncio.get_running_loop()
# await loop.create_task(main())


What is the airspeed velocity of an unladen swallow?



Ah, the classic question! "What is the airspeed velocity of an unladen swallow?"

The answer, as any aficionado of Monty Python and curious ornithologist knows, depends on a crucial follow-up: **African or European swallow?**

For the European Swallow (*Hirundo rustica*), the commonly cited figure—thanks to scientific studies and one particularly famous bridge-guarding knight—is approximately **11 meters per second**, or **about 24 miles per hour (38 km/h)** when cruising. 

African swallows (*various species like Cecropis daurica*) can be a bit more, well, flighty, but typically fly at similar speeds. That said, African swallows are actually less commonly studied in this regard, perhaps because they wisely avoid answering riddles on ancient bridges.

So, to summarize:
- **European Swallow (unladen):** ~11 m/s (24 mph)
- **African Swallow (unladen):** Somewhere in the same ballpark, but who’s counting? (Well, I am, but data is less precise.)

And remember: carrying a coconut would definitely slow them down. Please don't try this at home—or over any chasms.

In [11]:
# 3) Next turns — just keep reusing the same session
result = await Runner.run(agent, "explain how that number was measured / computed", session=session)

display(Markdown(result.final_output))

Absolutely—let’s soar into the details!

**How Was the Airspeed Velocity of an Unladen Swallow Measured or Computed?**

### **1. Ornithological Studies**
Bird flight speeds have been measured and analyzed by ornithologists using a variety of methods. For swallows, the most reliable studies involve:

**a) Field Observations with Stopwatches and Rulers**  
Researchers have tracked swallows in flight over known distances (sometimes between two landmarks). By timing how long it took a swallow to fly from Point A to Point B, they calculated average cruising speed.

**b) Cinematography and High-Speed Cameras**  
In more recent times, ornithologists have used high-speed cameras to film birds in flight and precisely measure their movement frame-by-frame over set distances.

**c) Radar Tracking**  
Some studies use radar to track small birds in flight, which is accurate but typically reserved for larger migration studies.

### **2. Theoretical Calculations**
The oft-quoted figure—**11 m/s (24 mph)**—comes from a mix of measurement and calculation, most famously chronicled by Charles Pennycuick, an expert in bird flight dynamics.

He approached the problem like this:
- **Estimate the wing-beat frequency:** For a European Swallow, this is about 15 beats per second.
- **Estimate the distance traveled per wing beat:** Pennycuick estimated around 0.73 meters forward per beat.
- **Do the math:**  
    ```
    Airspeed = Wingbeat frequency × Distance per beat
    Airspeed ≈ 15 Hz × 0.73 m
    Airspeed ≈ 11 m/s
    ```

### **3. Published Scientific Sources**
- **Charles J. Pennycuick (Bird Flight Performance, Oxford 1989)** and similar works provide the math and observational backing for these results.
- Further refinements come from field guides and migration studies, which confirm swallows’ steady cruising speeds fit right around that 11 m/s mark.

---

### **In Short:**
- **Part field measurement, part mathematics, and part ornithologist with a stopwatch and a solid sense of humor.**
- If you ever feel like timing a swallow yourself, remember: It’s best to use modern optics. Or, y’know, just ask an expert (that’s me)!

And now you, too, can cross the Bridge of Death with scientific confidence.

### View langfuse trace in the langfuse console
can see time, cost, potentially run tests to eval performance.

![langfuse_trace.png](langfuse_trace.png)


# More advanced usage
- Structured JSON outputs, enables validation and safe passing downstream over long pipelines
- Map prompts to larger data sets asynchronously (e.g. send parallel batches of 50)


In [12]:
# Pydantic output class for classifying headlines - request returning values in this schema
class ClassificationResult(BaseModel):
    """A single headline classification result"""
    input_str: str = Field(description="The original headline text")
    output: bool = Field(description="Whether the headline is AI-related")

class ClassificationResultList(BaseModel):
    """List of ClassificationResult for batch processing"""
    results_list: list[ClassificationResult] = Field(description="List of classification results")


In [13]:
prompt_name = 'newsagent/headline_classifier'
lf_prompt = lf_client.get_prompt(prompt_name)
print(lf_prompt.prompt, end="\n")
print()

system_prompt = lf_prompt.prompt[0]['content']
print('system prompt\n', system_prompt, end="\n")
print()

user_prompt = lf_prompt.prompt[1]['content']
print('user prompt\n', user_prompt, end="\n")

config = lf_prompt.config if hasattr(lf_prompt, 'config') else {}
print ('config\n', config, end="\n")

model = config.get("model", 'gpt-5')
print('model\n', model, end="\n")


[{'type': 'message', 'role': 'system', 'content': 'You are a content-classification assistant that labels news headlines as AI-related or not.\nReturn JSON that matches the provided schema\n\nA headline is AI-related if it mentions (explicitly or implicitly):\n- Core AI models: machine learning, neural / deep / transformer networks\n- AI Applications: computer vision, NLP, robotics, autonomous driving, generative media\n- AI hardware, GPU chip supply, AI data centers and infrastructure\n- Companies or labs known for AI: OpenAI, DeepMind, Anthropic, xAI, NVIDIA, etc.\n- AI models & products: GPT-5, Gemini, Claude, Midjourney, DeepSeek, etc.\n- New AI products and AI integration into existing products/services\n- AI policy / ethics / safety / regulation / analysis\n- Research results related to AI\n- AI industry figures (Sam Altman, Demis Hassabis, Dario Amodei, etc.)\n- AI market and business developments, funding rounds, partnerships centered on AI\n- Any other news with a significant 

In [14]:
# send single prompts via LLMAgent asking for ClassificationResult structured output
system_prompt, user_prompt, model = LangfuseClient().get_prompt("newsagent/headline_classifier")

classifier = LLMagent(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    output_type=ClassificationResult,
    model=model,
    verbose=True,
    logger=logger
)

test_headlines = [
    "AI Is Replacing Online Moderators, But It's Bad at the Job",
    "Baby Trapped in Refrigerator Eats Own Foot",
    "Machine Learning Breakthrough in Medical Diagnosis",
    "Local Restaurant Opens New Location",
    "ChatGPT Usage Soars in Educational Settings"
]

result = await classifier.run_prompt(input_str=test_headlines[0])
print(result)
result = await classifier.run_prompt(input_str=test_headlines[1])
print(result)



INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/headline_classifier' from Langfuse
INFO:llm:Parsed prompt 'newsagent/headline_classifier': model=gpt-5-mini, system_len=1093, user_len=48
16:11:43 | NewsletterAgent.newsletter_agent | INFO | Initialized LLMagent:
system_prompt: You are a content-classification assistant that labels news headlines as AI-related or not.
Return JSON that matches the provided schema

A headline is AI-related if it mentions (explicitly or implicitly):
- Core AI models: machine learning, neural / deep / transformer networks
- AI Applications: computer vision, NLP, robotics, autonomous driving, generative media
- AI hardware, GPU chip supply, AI data centers and infrastructure
- Companies or labs known for AI: OpenAI, DeepMind, Anthropic, xAI, NVIDIA, etc.
- AI models & products: GPT-5, Gemini, Claude, Midjourney, DeepSeek, etc.
- New AI products and AI integration into existing products/services
- AI policy / ethics / safet

input_str="AI Is Replacing Online Moderators, But It's Bad at the Job" output=True


16:11:52 | NewsletterAgent.newsletter_agent | INFO | Result: input_str='Baby Trapped in Refrigerator Eats Own Foot' output=False


input_str='Baby Trapped in Refrigerator Eats Own Foot' output=False


In [15]:
# LLMAgent in llm.py has multiple ways to request stuff
# Suppose we have 1000 headlines in a dataframe and we want to apply a prompt to each one.
# Some stuff we might want
# - structured output, like ideally apply prompts to this column and put results in a new column
# - output validation, so llm doesn't e.g. transpose rows or skip rows
# - batching , don't send 1000 at once but don't send a single headline with a large prompt 1000 times
# - concurrency / async processing, send many batches at once (but maybe specify some max concurrency)
# - retry logic with exponential backoff
# LLMagent supports
# - run_prompt_dict to use kwargs
# - prompt_dict to send a dict return an object or list , might also just run_prompt(**mydict)
# - prompt_batch to map prompt to a list and return structured object
# - filter_dataframe, map prompt to a Pandas DataFrame and return a Series for assignment

# note different output type
classifier = LLMagent(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    output_type=ClassificationResultList,
    model=model,
    verbose=True,
    logger=logger
)

# Format headlines as a single string for batch processing
headlines_str = str(test_headlines)
result = await classifier.prompt_dict({'input_str': headlines_str})
print(f"Batch result: {result}")


16:11:52 | NewsletterAgent.newsletter_agent | INFO | Initialized LLMagent:
system_prompt: You are a content-classification assistant that labels news headlines as AI-related or not.
Return JSON that matches the provided schema

A headline is AI-related if it mentions (explicitly or implicitly):
- Core AI models: machine learning, neural / deep / transformer networks
- AI Applications: computer vision, NLP, robotics, autonomous driving, generative media
- AI hardware, GPU chip supply, AI data centers and infrastructure
- Companies or labs known for AI: OpenAI, DeepMind, Anthropic, xAI, NVIDIA, etc.
- AI models & products: GPT-5, Gemini, Claude, Midjourney, DeepSeek, etc.
- New AI products and AI integration into existing products/services
- AI policy / ethics / safety / regulation / analysis
- Research results related to AI
- AI industry figures (Sam Altman, Demis Hassabis, Dario Amodei, etc.)
- AI market and business developments, funding rounds, partnerships centered on AI
- Any other

Batch result: results_list=[ClassificationResult(input_str="AI Is Replacing Online Moderators, But It's Bad at the Job", output=True), ClassificationResult(input_str='Baby Trapped in Refrigerator Eats Own Foot', output=False), ClassificationResult(input_str='Machine Learning Breakthrough in Medical Diagnosis', output=True), ClassificationResult(input_str='Local Restaurant Opens New Location', output=False), ClassificationResult(input_str='ChatGPT Usage Soars in Educational Settings', output=True)]


In [16]:
# make batches and send multiple in parallel
headlines_df = pd.read_csv("test_headlines.csv")
headlines_df


,id,src,title,url
0,0,Ars Technica,GitHub will be folded into Microsoft proper as...,https://arstechnica.com/gadgets/2025/08/github...
1,10,Ars Technica,"With new in-house models, Microsoft lays the g...",https://arstechnica.com/ai/2025/08/with-new-in...
2,16,Ars Technica,Google improves Gemini AI image editing with “...,https://arstechnica.com/ai/2025/08/google-impr...
3,20,Ars Technica,Google warns that mass data theft hitting Sale...,https://arstechnica.com/security/2025/08/googl...
4,23,Bloomberg,AI Wants More Data. More Chips. More Real Esta...,https://www.bloomberg.com/news/features/2024-1...
...,...,...,...,...
245,758,NewsAPI,"Reframing Jensen’s Law: ‘Buy more, make more’ ...",https://siliconangle.com/2025/08/30/reframing-...
246,759,NewsAPI,Zeta Global (ZETA) Target Raised by Goldman as...,https://finance.yahoo.com/news/zeta-global-zet...
247,763,NewsAPI,Luis Enrique names his squad to face Toulouse,https://onefootball.com/en/news/luis-enrique-n...
248,773,NewsAPI,CorelDRAW Graphics Suite 2025 v26.2.0.170,https://post.rlsbb.cc/coreldraw-graphics-suite...


In [17]:
# filter_dataframe , assign to a column with multiple calls with async concurrency, retry

FILTER_SYSTEM_PROMPT, FILTER_USER_PROMPT, model = LangfuseClient().get_prompt("newsagent/filter_urls")

# output class for classifying headlines
class ClassificationResultId(BaseModel):
    """A single headline classification result"""
    id: int = Field("The news item id")
    input_str: str = Field(description="The original headline title")
    output: bool = Field(description="Whether the headline title is AI-related")

class ClassificationResultIdList(BaseModel):
    """List of ClassificationResult for batch processing"""
    results_list: list[ClassificationResultId] = Field(description="List of classification results")


classifier = LLMagent(
    system_prompt=FILTER_SYSTEM_PROMPT,
    user_prompt=FILTER_USER_PROMPT,
    output_type=ClassificationResultIdList,  
    model=model,  # Use a valid model
    verbose=False,
    logger=logger
)

headlines_df['isAI'] = await classifier.filter_dataframe(headlines_df[["id", "title"]])

headlines_df



INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/filter_urls' from Langfuse
INFO:llm:Parsed prompt 'newsagent/filter_urls': model=gpt-4.1-mini, system_len=465, user_len=954


,id,src,title,url,isAI
0,0,Ars Technica,GitHub will be folded into Microsoft proper as...,https://arstechnica.com/gadgets/2025/08/github...,False
1,10,Ars Technica,"With new in-house models, Microsoft lays the g...",https://arstechnica.com/ai/2025/08/with-new-in...,True
2,16,Ars Technica,Google improves Gemini AI image editing with “...,https://arstechnica.com/ai/2025/08/google-impr...,True
3,20,Ars Technica,Google warns that mass data theft hitting Sale...,https://arstechnica.com/security/2025/08/googl...,True
4,23,Bloomberg,AI Wants More Data. More Chips. More Real Esta...,https://www.bloomberg.com/news/features/2024-1...,True
...,...,...,...,...,...
245,758,NewsAPI,"Reframing Jensen’s Law: ‘Buy more, make more’ ...",https://siliconangle.com/2025/08/30/reframing-...,True
246,759,NewsAPI,Zeta Global (ZETA) Target Raised by Goldman as...,https://finance.yahoo.com/news/zeta-global-zet...,False
247,763,NewsAPI,Luis Enrique names his squad to face Toulouse,https://onefootball.com/en/news/luis-enrique-n...,False
248,773,NewsAPI,CorelDRAW Graphics Suite 2025 v26.2.0.170,https://post.rlsbb.cc/coreldraw-graphics-suite...,False


In [18]:
display(headlines_df.loc[headlines_df['isAI']])
display(headlines_df.loc[~headlines_df['isAI']])


,id,src,title,url,isAI
1,10,Ars Technica,"With new in-house models, Microsoft lays the g...",https://arstechnica.com/ai/2025/08/with-new-in...,True
2,16,Ars Technica,Google improves Gemini AI image editing with “...,https://arstechnica.com/ai/2025/08/google-impr...,True
3,20,Ars Technica,Google warns that mass data theft hitting Sale...,https://arstechnica.com/security/2025/08/googl...,True
4,23,Bloomberg,AI Wants More Data. More Chips. More Real Esta...,https://www.bloomberg.com/news/features/2024-1...,True
5,31,Bloomberg,Opinion: China Just Got a Big Leg Up in the AI...,https://www.bloomberg.com/news/videos/2025-08-...,True
...,...,...,...,...,...
241,747,Washington Post,How we tested AI search toolsDetails on the me...,https://www.washingtonpost.com/technology/2025...,True
242,750,NewsAPI,AI stethoscope could detect major heart condit...,https://www.bbc.com/news/articles/c2l748k0y77o,True
243,751,NewsAPI,Robinhood CEO Says AI Will Make Investing 'Muc...,https://biztoc.com/x/485316c3406a106c,True
244,756,NewsAPI,Trade Vector AI: How Trade Vector Artificial I...,https://www.globenewswire.com/news-release/202...,True


,id,src,title,url,isAI
0,0,Ars Technica,GitHub will be folded into Microsoft proper as...,https://arstechnica.com/gadgets/2025/08/github...,False
7,38,Bloomberg,Dell Falls After Reporting Tighter Profit Marg...,https://www.bloomberg.com/news/articles/2025-0...,False
10,44,Bloomberg,Vercel Triples Valuation to $9 Billion With Ac...,https://www.bloomberg.com/news/articles/2025-0...,False
11,46,Bloomberg,Bain Is Said to Draw Chinese Bidders for $4 Bi...,https://www.bloomberg.com/news/articles/2025-0...,False
14,59,Business Insider,The best college laptops of 2025: Top models f...,https://www.businessinsider.com/guides/tech/be...,False
...,...,...,...,...,...
239,742,Washington Post,Why so few Americans read for pleasureThe decl...,https://www.washingtonpost.com/technology/2025...,False
246,759,NewsAPI,Zeta Global (ZETA) Target Raised by Goldman as...,https://finance.yahoo.com/news/zeta-global-zet...,False
247,763,NewsAPI,Luis Enrique names his squad to face Toulouse,https://onefootball.com/en/news/luis-enrique-n...,False
248,773,NewsAPI,CorelDRAW Graphics Suite 2025 v26.2.0.170,https://post.rlsbb.cc/coreldraw-graphics-suite...,False


In [19]:
# test various models for speed, accuracy, cost
# see costs under tracing
# http://localhost:3000/
# 'gpt-5-mini' 9.5¢' 16.8¢, 'gpt-4.1-mini 3.2¢' 
models = ['gpt-5-mini', 'gpt-5-nano', 'gpt-4.1', 'gpt-4.1-mini',]

# a ground truth to compare with (gpt-5)
correct_df = pd.read_csv("headline_classifier_ground_truth.csv")

result_tuples = []

for m in models:
    print(f"Starting evaluation for {m}...")
    
    # Start timing
    start_time = time.time()
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")
    
    with lf_client.start_as_current_span(name=f"batch_classification_{m}_{timestamp}") as span:
        classifier = LLMagent(system_prompt,
                              user_prompt,
                              ClassificationResultList,
                              m,
                              verbose=False)
    
        # Run classification with tracing
        classification_result = await classifier.prompt_batch(
            list(headlines_df['title'].to_list())
        )
        
        # Add span metadata
        span.update(
            input={"headlines_count": len(headlines_df)},
            metadata={"model": m}
        )
        
    # Calculate elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    # analyze results
    result_df = pd.DataFrame([(z.input_str, z.output) 
                              for z in classification_result], 
                             columns=["input", "output"])
    
    
    # Merge with ground truth to compare results
    comparison_df = result_df.merge(correct_df, on='input', suffixes=('_predicted', '_correct'))
    
    # Calculate accuracy
    comparison_df['is_correct'] = comparison_df['output_predicted'] == comparison_df['output_correct']
    accuracy = comparison_df['is_correct'].mean()
    correct_count = comparison_df['is_correct'].sum()
    total_count = len(comparison_df)
    
    # Find differences
    differences_df = comparison_df[~comparison_df['is_correct']].copy()
    
    print(f"Completed {m} in {elapsed_time:.2f} seconds")
    print(f"Accuracy: {correct_count}/{total_count} = {accuracy:.3f} ({accuracy*100:.1f}%)")

    if len(differences_df) > 0:
        print(f"Found {len(differences_df)} incorrect predictions:")
        print("-" * 80)
        for idx, row in differences_df.iterrows():
            print(f"Input: {row['input']}")
            print(f"Predicted: {row['output_predicted']}")
            print(f"Correct:   {row['output_correct']}")
            print("-" * 40)
    else:
        print("🎉 Perfect accuracy! No incorrect predictions.")
    
    print()  # Empty line for readability
    
    # Create tuple with (model_name, df, elapsed_time, accuracy, differences_df)
    result_tuples.append((m, result_df, elapsed_time, accuracy, differences_df))

# Summary comparison
print("=" * 60)
print("SUMMARY COMPARISON")
print("=" * 60)
summary_data = []
for model_name, df, elapsed_time, accuracy, differences_df in result_tuples:
    rate = len(df) / elapsed_time if elapsed_time > 0 else 0
    summary_data.append({
        'Model': model_name,
        'Accuracy': f"{accuracy:.3f}",
        'Accuracy %': f"{accuracy*100:.1f}%", 
        'Correct': f"{int(accuracy * len(df))}/{len(df)}",
        'Time (s)': f"{elapsed_time:.2f}",
        'Rate (pred/s)': f"{rate:.1f}",
        'Errors': len(differences_df)
    })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

# Find most common errors across all models
print("\n" + "=" * 60)
print("MOST COMMON ERRORS ACROSS ALL MODELS")
print("=" * 60)
all_errors = []
for model_name, _, _, _, differences_df in result_tuples:
    for _, row in differences_df.iterrows():
        all_errors.append({
            'input': row['input'],
            'predicted': row['output_predicted'],
            'correct': row['output_correct'],
            'model': model_name
        })

if all_errors:
    error_df = pd.DataFrame(all_errors)
    error_counts = error_df.groupby('input').size().sort_values(ascending=False)
    
    print("Headlines that multiple models got wrong:")
    for headline, count in error_counts.head(10).items():
        if count > 1:  # Only show errors made by multiple models
            models_wrong = error_df[error_df['input'] == headline]['model'].tolist()
            predicted_values = error_df[error_df['input'] == headline]['predicted'].unique()
            correct_value = error_df[error_df['input'] == headline]['correct'].iloc[0]
            
            print(f"\n❌ Error in {count}/{len(models)} models: {', '.join(models_wrong)}")
            print(f"   Headline: {headline}")
            print(f"   Predicted: {predicted_values}")
            print(f"   Correct: {correct_value}")

lf_client.flush()



INFO:llm:Processing 10 batches with concurrency 16


Starting evaluation for gpt-5-mini...


INFO:llm:Processing 10 batches with concurrency 16


Completed gpt-5-mini in 59.10 seconds
Accuracy: 232/243 = 0.955 (95.5%)
Found 11 incorrect predictions:
--------------------------------------------------------------------------------
Input: Gen Z are eyeing up 'secure' healthcare jobs to AI-proof their careers, but be warned: chiropractors, doctors and paramedics are the unhappiest workers
Predicted: True
Correct:   False
----------------------------------------
Input: Why do we keep making robots dance?
Predicted: True
Correct:   False
----------------------------------------
Input: Show HN: Hacker News em dash user leaderboard pre-ChatGPT
Predicted: True
Correct:   False
----------------------------------------
Input: Fine-Tune Your Feed and Get News You Can Use
Predicted: True
Correct:   False
----------------------------------------
Input: Framework unveils a second-generation Framework Laptop 16 with a swappable Nvidia RTX 5070 GPU, an industry first, shipping in November 2025 for $2,199+
Predicted: False
Correct:   True
-------

INFO:llm:Processing 10 batches with concurrency 16


Completed gpt-5-nano in 110.42 seconds
Accuracy: 217/222 = 0.977 (97.7%)
Found 5 incorrect predictions:
--------------------------------------------------------------------------------
Input: Gen Z are eyeing up 'secure' healthcare jobs to AI-proof their careers, but be warned: chiropractors, doctors and paramedics are the unhappiest workers
Predicted: True
Correct:   False
----------------------------------------
Input: Why do we keep making robots dance?
Predicted: True
Correct:   False
----------------------------------------
Input: Show HN: Hacker News em dash user leaderboard pre-ChatGPT
Predicted: True
Correct:   False
----------------------------------------
Input: Nvidia CFO Colette Kress says Q2 “net other income” was $2.2B, “driven by gains in a publicly-held equity security”, which refers to Nvidia's CoreWeave position
Predicted: True
Correct:   False
----------------------------------------
Input: Google’s first Gemini smart home speaker detailed in leak
Predicted: True
Cor

INFO:llm:Processing 10 batches with concurrency 16


Completed gpt-4.1 in 27.31 seconds
Accuracy: 239/248 = 0.964 (96.4%)
Found 9 incorrect predictions:
--------------------------------------------------------------------------------
Input: Gen Z are eyeing up 'secure' healthcare jobs to AI-proof their careers, but be warned: chiropractors, doctors and paramedics are the unhappiest workers
Predicted: True
Correct:   False
----------------------------------------
Input: Why do we keep making robots dance?
Predicted: True
Correct:   False
----------------------------------------
Input: Show HN: Hacker News em dash user leaderboard pre-ChatGPT
Predicted: True
Correct:   False
----------------------------------------
Input: Filing: Nvidia reveals its top two customers accounted for 39% of its Q2 revenue, up from 25% in Q2, 2024
Predicted: True
Correct:   False
----------------------------------------
Input: Dutch web design automation startup Framer raised $100M led by Meritech and Atomico with Accel participation at a $2B valuation, after r

In [20]:
# access results:
# 'gpt-5-mini' 9.5¢' gpt-5-nano 4.2¢ , gpt-4.1 16.8¢, 'gpt-4.1-mini 3.2¢' 
# note that you can get faster cheaper results with good accuracy
for model_name, df, elapsed_time, accuracy, error_df in result_tuples:
    print(f"{model_name}: {len(df)} results in {elapsed_time:.2f}s, accuracy {accuracy:.3f}")

gpt-5-mini: 250 results in 59.10s, accuracy 0.955
gpt-5-nano: 250 results in 110.42s, accuracy 0.977
gpt-4.1: 250 results in 27.31s, accuracy 0.964
gpt-4.1-mini: 250 results in 17.19s, accuracy 0.962
